In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

# PostgreSQL JDBC URL
POSTGRES_URL = "jdbc:postgresql://localhost:5432/test_user"
POSTGRES_PROPERTIES = {
    "user": "test_user",
    "password": "testpassword",
    "driver": "org.postgresql.Driver"
}

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("PySpark-Postgres") \
    .config('spark.jars',"path/to/notebooks/jars/postgresql-42.7.5.jar") \
    .getOrCreate()

#spark.sparkContext.setLogLevel("ERROR")
spark

In [2]:
# Read data from PostgreSQL
df = spark.read \
    .format("jdbc") \
    .option("url", POSTGRES_URL) \
    .option("dbtable", "employees") \
    .option("user", POSTGRES_PROPERTIES["user"]) \
    .option("password", POSTGRES_PROPERTIES["password"]) \
    .option("driver", POSTGRES_PROPERTIES["driver"]) \
    .load()

print("Original Data:")
df.show()


Original Data:
+---+------+---+
| id|  name|age|
+---+------+---+
|  1|Name_A| 30|
|  2|Name_B| 25|
|  3|Name_C| 35|
+---+------+---+



In [3]:
# Modify Data (Add a new column)
df_transformed = df.withColumn("bonus", col("age") * lit(100))

print("Modified Data:")
df_transformed.show()

Modified Data:
+---+------+---+-----+
| id|  name|age|bonus|
+---+------+---+-----+
|  1|Name_A| 30| 3000|
|  2|Name_B| 25| 2500|
|  3|Name_C| 35| 3500|
+---+------+---+-----+



In [4]:
# Write back to PostgreSQL (New Table)
df_transformed.write \
    .format("jdbc") \
    .option("url", POSTGRES_URL) \
    .option("dbtable", "employees_bonus") \
    .option("user", POSTGRES_PROPERTIES["user"]) \
    .option("password", POSTGRES_PROPERTIES["password"]) \
    .option("driver", POSTGRES_PROPERTIES["driver"]) \
    .mode("overwrite") \
    .save()

print("Data successfully written to PostgreSQL!")

# Stop Spark Session
spark.stop()

Data successfully written to PostgreSQL!
